# Dataset creation

![flowchart](./resources/dataset_creation_flowchart.drawio.png)


In [1]:
import pandas as pd
import numpy as np

from utils.file_utils import open_json, write_json
from utils.dataset_creation import *
from utils.dataset_mapping import *


In [2]:
FRESH_START = True
UPDATE_MAPPING = False


In [3]:
COLUMNS = ["pdbs", "uniprot", "wild_aa", "mutation_position",
           "mutated_aa", "pH",
           "sequence", "length", "chain_start", "chain_end",
           "AlphaFoldDB", "Tm", "ddG", "dTm", 
           "dataset_source", "infos_found"]

OUPUT_DIR = "./data/main_dataset_creation"
LOCAL_UNIPROT_INFOS_PATH = OUPUT_DIR+"/uniprot_infos.json"
PDB_UNIPROT_MAPPING_PATH = OUPUT_DIR+"/mapping/pdb_uniprot_mapping.json"
LINKED_UNIPROT_MAPPING_PATH = OUPUT_DIR+"/mapping/linked_uniprot_mapping.json"
SEQUENCE_UNIPROT_MAPPING_PATH = OUPUT_DIR+"/mapping/sequence_uniprot_mapping.json"
PDB_NO_UNIPROT_PATH = OUPUT_DIR+"/mapping/pdb_no_uniprot.json"
SEQUENCE_NO_UNIPROT_PATH = OUPUT_DIR+"/mapping/sequence_no_uniprot.json"
DATASET_OUTPUT_PATH = OUPUT_DIR+"/dataset.csv"


# Infos for dataset creation


In [4]:
local_uniprot_infos = open_json(LOCAL_UNIPROT_INFOS_PATH)
dataset_config = open_json(OUPUT_DIR+"/dataset_config.json")

print(f"loaded {len(local_uniprot_infos)} uniprot infos from local storage")
errors = {
    "no_sequence_in_data": 0,
    "not_in_local": 0,
    "wrong_position": 0,
    "no_uniprot": 0,
    "no_pdb": 0,
    "no_sequence": 0,
}


loaded 472 uniprot infos from local storage


In [5]:
df = pd.read_csv("./data/ProThermDB/processed_prothermdb.csv")
df.mutation_code.isna().sum()


0

##### FireProtDB


In [6]:
if FRESH_START:
    main_df = pd.DataFrame()
    main_df = add_missing_column(main_df, COLUMNS)
else:
    main_df = pd.read_csv(DATASET_OUTPUT_PATH)

for dataset_source in dataset_config["dataset_to_process"]:
    individual_config = dataset_config[dataset_source]
    # load csv
    if dataset_source == "thermomutdb":
        df = pd.read_json(individual_config["data_path"])
        df = df[df.mut_count.eq(0)]
        df[df.uniprot.eq('-')] = np.nan
        print(len(df))
    else:
        df = pd.read_csv(individual_config["data_path"])
    # rename columns
    df.rename(columns=individual_config["renaming_dict"],
              inplace=True)
    # add missing columns
    df = add_missing_column(df, COLUMNS)
    # split mutation code if needed
    if individual_config["need_mutation_code_split"]:
        df = df.apply(apply_split_mutation_code, axis=1)
    # keep only COLUMNS
    df = df[COLUMNS]
    # drop duplicates
    df.drop_duplicates(inplace=True)
    # add dataset_source
    df["dataset_source"] = dataset_source
    # index start at 0
    df["mutation_position"] = df["mutation_position"].apply(lambda x: x-1)
    # apply target corrections
    df["ddG"] = df["ddG"].apply(
        lambda x: x*individual_config["corrections"]["ddG"])
    df["dTm"] = df["dTm"].apply(
        lambda x: x*individual_config["corrections"]["dTm"])
    # better to initialize infos_found at False than nan
    df["infos_found"] = False
    
    # check number of rows without uniprot
    # check validity of uniprot, and add the infos for those
    df = df.apply(lambda row: apply_valid_uniprot(
        row, local_uniprot_infos, dataset_config, errors), axis=1)
    
    
    print(f"processed {dataset_source}:")
    print(f"{errors=}")
    print(f"{len(df)=}, number of rows without infos: {df.infos_found.isna().sum()}")

    main_df = pd.concat([main_df, df], ignore_index=True)

# removing duplicates
print(len(main_df))
main_df.drop_duplicates(COLUMNS[1:-2], inplace=True)
print(len(main_df))

# save
write_json(LOCAL_UNIPROT_INFOS_PATH, local_uniprot_infos)
main_df.to_csv(DATASET_OUTPUT_PATH, index=False)


processed fireprotdb:
errors={'no_sequence_in_data': 211, 'not_in_local': 211, 'wrong_position': 11, 'no_uniprot': 0, 'no_pdb': 0, 'no_sequence': 0}
len(df)=9934, number of rows without infos: 0
11201


/tmp/ipykernel_18432/357627962.py:51: FutureWarning: Behavior when concatenating bool-dtype and numeric-dtype arrays is deprecated; in a future version these will cast to object dtype (instead of coercing bools to numeric values). To retain the old behavior, explicitly cast bool-dtype arrays to numeric dtype.
  main_df = pd.concat([main_df, df], ignore_index=True)


error, positions are not all int
mutation_position=nan, position_offset=1, chain_start=0
sequence='MQRGKVKWFNNEKGYGFIEVEGGSDVFVHFTAIQGEGFKTLEEGQEVSFEIVQGNRGPQAANVVKL'
error, positions are not all int
mutation_position=nan, position_offset=1, chain_start=0
sequence='MQRGKVKWFNNEKGYGFIEVEGGSDVFVHFTAIQGEGFKTLEEGQEVSFEIVQGNRGPQAANVVKL'
error, positions are not all int
mutation_position=nan, position_offset=1, chain_start=0
sequence='MELKHSISDYTEAEFLQLVTTICNADTSSEEELVKLVTHFEEMTEHPSGSDLIYYPKEGDDDSPSGIVNTVKQWRAANGKSGFKQG'
Exception: HTTP Error 400: Bad Request
uniprot: GQ884175, type: <class 'str'>
Exception: HTTP Error 400: Bad Request
uniprot: GQ884175, type: <class 'str'>
processed thermomutdb:
errors={'no_sequence_in_data': 229, 'not_in_local': 229, 'wrong_position': 2647, 'no_uniprot': 0, 'no_pdb': 0, 'no_sequence': 0}
len(df)=10946, number of rows without infos: 0
processed O2567_new:
errors={'no_sequence_in_data': 229, 'not_in_local': 229, 'wrong_position': 2647, 'no_uniprot': 0, 'no_p

In [7]:
# print(len(main_df))
# main_df.drop_duplicates(COLUMNS[1:-2], inplace=True)
# print(len(main_df))
# main_df.head(1)


### update mapping and try to add infos


In [8]:
if not UPDATE_MAPPING:
    # don't go beyond here with Run All
    assert False


AssertionError: 

In [9]:
# update pdb to uniprot mapping
update_pdb_uniprot_mapping(LOCAL_UNIPROT_INFOS_PATH,
                           PDB_UNIPROT_MAPPING_PATH,
                           LINKED_UNIPROT_MAPPING_PATH)

pdb_uniprot_mapping = open_json(PDB_UNIPROT_MAPPING_PATH)
linked_uniprot_mapping = open_json(LINKED_UNIPROT_MAPPING_PATH)
pdb_without_uniprot = open_json(PDB_NO_UNIPROT_PATH)


added 1777 entries to pdb_uniprot_mapping


In [10]:
# add infos based on pdb not uniprot
df = pd.read_csv(DATASET_OUTPUT_PATH)

with_infos = df.infos_found.sum()
df = df.apply(lambda row: apply_infos_from_pdb(row, local_uniprot_infos, pdb_uniprot_mapping,
                                               linked_uniprot_mapping, dataset_config,
                                               pdb_without_uniprot, errors),
              axis=1)
print(f"added {df.infos_found.sum()-with_infos} new infos thanks to uniprot_from_pdb")
df.to_csv(DATASET_OUTPUT_PATH, index=False)


error, positions are not all int
mutation_position=nan, position_offset=1, chain_start=0
sequence='MQRGKVKWFNNEKGYGFIEVEGGSDVFVHFTAIQGEGFKTLEEGQEVSFEIVQGNRGPQAANVVKL'
error, positions are not all int
mutation_position=nan, position_offset=1, chain_start=0
sequence='MQRGKVKWFNNEKGYGFIEVEGGSDVFVHFTAIQGEGFKTLEEGQEVSFEIVQGNRGPQAANVVKL'
error, positions are not all int
mutation_position=nan, position_offset=1, chain_start=0
sequence='MELKHSISDYTEAEFLQLVTTICNADTSSEEELVKLVTHFEEMTEHPSGSDLIYYPKEGDDDSPSGIVNTVKQWRAANGKSGFKQG'
error, positions are not all int
mutation_position=nan, position_offset=0, chain_start=0
sequence='MNLVLMGLPGAGKGTQGERIVEDYGIPHISTGDMFRAAMKEETPLGLEAKSYIDKGELVPDEVTIGIVKERLGKDDCERGFLLDGFPRTVAQAEALEEILEEYGKPIDYVINIEVDKDVLMERLTGRRICSVCGTTYHLVFNPPKTPGICDKDGGELYQRADDNEETVSKRLEVNMKQTQPLLDFYSEKGYLANVNGQQDIQDVYADVKDLLGGLKK'
error, positions are not all int
mutation_position=nan, position_offset=0, chain_start=0
sequence='MEQRITLKDYAMRFGQTKTAKDLGVYQSAINKAIHAGRKIFLTINADGSVYAEEVKPFPSNK

In [11]:
# update sequence to uniprot mapping
update_sequence_uniprot_mapping(LOCAL_UNIPROT_INFOS_PATH,
                                SEQUENCE_UNIPROT_MAPPING_PATH,
                                LINKED_UNIPROT_MAPPING_PATH)

sequence_uniprot_mapping = open_json(SEQUENCE_UNIPROT_MAPPING_PATH)
sequence_without_uniprot = open_json(SEQUENCE_NO_UNIPROT_PATH)


added 69 entries to sequence_uniprot_mapping


In [13]:
# add infos based on sequence not pdb or uniprot

df = pd.read_csv(DATASET_OUTPUT_PATH)

with_infos = df.infos_found.sum()
df = df.apply(lambda row: apply_infos_from_sequence(row, local_uniprot_infos, sequence_uniprot_mapping,
                                                    linked_uniprot_mapping, dataset_config,
                                                    sequence_without_uniprot, errors),
              axis=1)
# print(
#     f"added {df.infos_found.sum()-with_infos} new infos thanks to uniprot_from_sequence")

df.to_csv(DATASET_OUTPUT_PATH, index=False)


In [15]:
# make sure mapping and other data is saved
write_json(LOCAL_UNIPROT_INFOS_PATH, local_uniprot_infos)
write_json(PDB_NO_UNIPROT_PATH, pdb_without_uniprot)
write_json(SEQUENCE_NO_UNIPROT_PATH, sequence_without_uniprot)


In [16]:
df = df[COLUMNS[1:]]
df.to_csv("./data/main_dataset_creation/dataset_no_pdb.csv")

In [29]:
df = df[~df.infos_found.eq("0.0")]
print(len(df))
df.drop_duplicates(COLUMNS[1:-2], inplace=True)
print(len(df))


25699
25483


In [30]:
df.to_csv("./data/main_dataset_creation/dataset_no_pdb_with_infos.csv")


In [31]:
s="MMSFVSLLLVGILFHATQAEQLTKCEVFQKLKDLKDYGGVSLPEWVCTAFHTSGYDTQAIVQNNDSTEYGLFQINNKIWCKDDQNPHSRNICNISCDKFLDDDLTDDIVCAKKILDKVGINYWLAHKALCSEKLDQWLCEKL"
s[107]

'I'

In [32]:
df.pH.isna().sum()

878